# Data Analysis of Data Analysts

Project 1

In [ ]:
# Dependencies and Setup
import os
import csv
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from scipy.stats import sem
from scipy.stats import linregress
import scipy.stats as st
import numpy as np
import requests
import json

# Import the API key
from census import Census
from config import geoapify_key

# File to Load (Remember to Change These)
 = Path("../Resources/schools_complete.csv")
 = Path("../Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
 = pd.read_csv(school_data_to_load)
 = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  